In [368]:
import pandas as pd
import numpy as np
from keras.optimizers import Adam
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split

from tensorflow import keras

from keras.models import Sequential
from keras.layers import Dense, Dropout
# import keras_tuner

In [369]:
merged_data_path = '../data/merged_data.jsonl'
data = pd.read_json(merged_data_path, lines=True)

# data = data.drop(
#     columns=["release_date", "key", "loudness",
#              "explicit", "popularity", "duration_ms", "danceability", "energy", "speechiness",
#              "acousticness", "instrumentalness", "liveness", "valence", "tempo"])

data = data.drop(
    columns=["release_date", "name"])

data.head(500)

,user_id,popularity,duration_ms,explicit,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,favourite_genres,genres,skipped,number_of_matching_genres
0,101,34,247707,0,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.3410,0.669,123.025,"[permanent wave, mandopop, funk]","[album rock, art rock, classic rock, folk rock...",False,0
1,103,34,247707,0,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.3410,0.669,123.025,"[filmi, regional mexican, folk]","[album rock, art rock, classic rock, folk rock...",False,0
2,132,34,247707,0,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.3410,0.669,123.025,"[psychedelic rock, country rock, rock en espanol]","[album rock, art rock, classic rock, folk rock...",False,1
3,132,35,140067,0,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.3910,0.448,106.861,"[psychedelic rock, country rock, rock en espanol]","[album rock, art rock, classic rock, folk rock...",False,1
4,132,35,140067,0,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.3910,0.448,106.861,"[psychedelic rock, country rock, rock en espanol]","[album rock, art rock, classic rock, folk rock...",False,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,119,11,332558,0,0.326,0.735,9,-5.953,0.0369,0.41100,0.710000,0.1940,0.835,174.856,"[hard rock, alternative metal, singer-songwriter]","[album rock, art rock, blues, blues rock, brit...",False,1
496,101,23,423360,0,0.309,0.557,5,-7.049,0.0275,0.11900,0.000003,0.0583,0.205,174.372,"[permanent wave, mandopop, funk]","[album rock, art rock, classic rock, classic u...",True,1
497,101,23,423360,0,0.309,0.557,5,-7.049,0.0275,0.11900,0.000003,0.0583,0.205,174.372,"[permanent wave, mandopop, funk]","[album rock, art rock, classic rock, classic u...",True,1
498,103,23,423360,0,0.309,0.557,5,-7.049,0.0275,0.11900,0.000003,0.0583,0.205,174.372,"[filmi, regional mexican, folk]","[album rock, art rock, classic rock, classic u...",False,0


In [370]:
unique_favourite_genres = set()
for genres in data['favourite_genres']:
  for genre in genres:
    unique_favourite_genres.add(genre)

unique_genres = set()
for genres in data['genres']:
  for genre in genres:
    unique_genres.add(genre)

# show all common genres between all favourite_genres and genres
print("unique_favourite_genres", len(unique_favourite_genres))
print("unique_genres", len(unique_genres))
common_genres = unique_favourite_genres.intersection(unique_genres)
print("common genres", common_genres)

# remove genres that are not in common_genres
data['genres'] = data['genres'].apply(
    lambda x: [genre for genre in x if genre in common_genres])

data.head()

# TODO remove ??? moze niepotrzebne

unique_favourite_genres 46
unique_genres 1766
common genres {'blues rock', 'singer-songwriter', 'pop', 'album rock', 'quiet storm', 'ranchera', 'motown', 'vocal jazz', 'classic rock', 'soul', 'art rock', 'pop rock', 'folk', 'lounge', 'j-pop', 'filmi', 'hard rock', 'new wave', 'mellow gold', 'country rock', 'alternative rock', 'permanent wave', 'new romantic', 'new wave pop', 'latin alternative', 'soft rock', 'rock en espanol', 'c-pop', 'brill building pop', 'adult standards', 'alternative metal', 'mpb', 'turkish pop', 'funk', 'europop', 'hoerspiel', 'latin', 'latin pop', 'latin rock', 'tropical', 'mandopop', 'regional mexican', 'metal', 'psychedelic rock', 'rock', 'dance pop'}


,user_id,popularity,duration_ms,explicit,danceability,energy,key,loudness,speechiness,acousticness,instrumentalness,liveness,valence,tempo,favourite_genres,genres,skipped,number_of_matching_genres
0,101,34,247707,0,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,"[permanent wave, mandopop, funk]","[album rock, art rock, classic rock, psychedel...",False,0
1,103,34,247707,0,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,"[filmi, regional mexican, folk]","[album rock, art rock, classic rock, psychedel...",False,0
2,132,34,247707,0,0.491,0.606,7,-10.172,0.0377,0.00327,0.000008,0.341,0.669,123.025,"[psychedelic rock, country rock, rock en espanol]","[album rock, art rock, classic rock, psychedel...",False,1
3,132,35,140067,0,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,"[psychedelic rock, country rock, rock en espanol]","[album rock, art rock, classic rock, psychedel...",False,1
4,132,35,140067,0,0.449,0.749,9,-8.585,0.0775,0.01000,0.000000,0.391,0.448,106.861,"[psychedelic rock, country rock, rock en espanol]","[album rock, art rock, classic rock, psychedel...",False,1


In [371]:
# Combine genres and favourite_genres
all_genres = list(data['favourite_genres'] + data['genres'])

# One-hot encode the genres
mlb = MultiLabelBinarizer()
mlb.fit(all_genres)

encoded_all_genres = mlb.fit_transform(all_genres)

# Split encoded_genres into genres and favourite_genres
# encoded_favourite_genres = encoded_all_genres[:, :len(data['favourite_genres'][0])]
# encoded_genres = encoded_all_genres[:, len(data['favourite_genres'][0]):]

encoded_favourite_genres = mlb.transform(data['favourite_genres'])
encoded_genres = mlb.transform(data['genres'])

popularity_normalized = data['popularity'].values.reshape(-1, 1)
popularity_normalized = (popularity_normalized - np.min(popularity_normalized)) / (
    np.max(popularity_normalized) - np.min(popularity_normalized))

duration_ms_normalized = data['duration_ms'].values.reshape(-1, 1)
duration_ms_normalized = (duration_ms_normalized - np.min(duration_ms_normalized)) / (
    np.max(duration_ms_normalized) - np.min(duration_ms_normalized))

danceability_normalized = data['danceability'].values.reshape(-1, 1)
danceability_normalized = (danceability_normalized - np.min(danceability_normalized)) / (
    np.max(danceability_normalized) - np.min(danceability_normalized))

energy_normalized = data['energy'].values.reshape(-1, 1)
energy_normalized = (energy_normalized - np.min(energy_normalized)) / (
    np.max(energy_normalized) - np.min(energy_normalized))

number_of_matching_genres_normalized = data['number_of_matching_genres'].values.reshape(-1, 1)
number_of_matching_genres_normalized = (number_of_matching_genres_normalized - np.min(
    number_of_matching_genres_normalized)) / (np.max(number_of_matching_genres_normalized) - np.min(
    number_of_matching_genres_normalized))

# create data frame from data genres, data favourite_genres, encoded genres, encoded favourite_genres
df = pd.DataFrame(
    data={'popularity': data['popularity'],
          'popularity_normalized': popularity_normalized.reshape(-1),
          'duration_ms': data['duration_ms'],
          'duration_ms_normalized': duration_ms_normalized.reshape(-1),
          'number_of_matching_genres': data['number_of_matching_genres'],
          'number_of_matching_genres_normalized': number_of_matching_genres_normalized.reshape(-1),
          'skipped': data['skipped']})

df.head(5)

,popularity,popularity_normalized,duration_ms,duration_ms_normalized,number_of_matching_genres,number_of_matching_genres_normalized,skipped
0,34,0.354167,247707,0.092836,0,0.000000,False
1,34,0.354167,247707,0.092836,0,0.000000,False
2,34,0.354167,247707,0.092836,1,0.333333,False
3,35,0.364583,140067,0.046724,1,0.333333,False
4,35,0.364583,140067,0.046724,1,0.333333,False


In [372]:
# TODO uzywac keras tokenizer?

# Concatenate the one-hot encoded columns
# X = np.concatenate([encoded_favourite_genres, encoded_genres], axis=1)
X = np.concatenate(
    [popularity_normalized, duration_ms_normalized, danceability_normalized, energy_normalized,
     number_of_matching_genres_normalized], axis=1)

# create df from X
# df = pd.DataFrame(data=X)
# df.head(5)
#
# Extract the labels
y = data['skipped'].astype(int).values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# count skipped and not skipped songs in X_train
print("Y_train not skipped", np.count_nonzero(y_test == 0))
print("Y_train skipped", np.count_nonzero(y_test == 1))
print("Y_train skipped %", np.count_nonzero(y_test == 1) / len(y_test))


Y_train not skipped 1257
Y_train skipped 720
Y_train skipped % 0.36418816388467373


In [373]:
def build_model(hp):
  model = Sequential()
  model.add(Dense(32 * 2, activation='relu', input_shape=(X_train.shape[1],)))
  model.add(Dropout(0.5))
  model.add(Dense(32, activation='relu'))
  model.add(Dropout(0.5))
  # model.add(Dense(1000, activation='relu'))
  # model.add(Dropout(0.5))
  model.add(Dense(1, activation='sigmoid'))

  # model.compile(
  #     optimizer=keras.optimizers.Adam(hp.Choice('learning_date', values=[0.5, 0.1, 0.01])),
  #     loss='binary_crossentropy', metrics=['accuracy'])
  model.compile(
      optimizer=Adam(),
      loss='binary_crossentropy', metrics=['accuracy'])
  return model


In [374]:
epochs = 500
batch_size = 32

model = build_model(None)

history = model.fit(
    X_train,
    y_train,
    validation_data=(X_test, y_test),
    epochs=epochs,
    batch_size=batch_size,
    verbose=2
)

# tuner = keras_tuner.tuners.Hyperband(
#     build_model,
#     objective='val_accuracy',
#     max_epochs=50,
#     max_trials=10,
#     executions_per_trial=2,
#     directory='my_dir')


Epoch 1/500
247/247 - 2s - loss: 0.6651 - accuracy: 0.6261 - val_loss: 0.6526 - val_accuracy: 0.6358 - 2s/epoch - 8ms/step
Epoch 2/500
247/247 - 1s - loss: 0.6562 - accuracy: 0.6328 - val_loss: 0.6502 - val_accuracy: 0.6358 - 1s/epoch - 5ms/step
Epoch 3/500
247/247 - 1s - loss: 0.6546 - accuracy: 0.6335 - val_loss: 0.6506 - val_accuracy: 0.6358 - 1s/epoch - 5ms/step
Epoch 4/500
247/247 - 1s - loss: 0.6543 - accuracy: 0.6335 - val_loss: 0.6490 - val_accuracy: 0.6358 - 1s/epoch - 5ms/step
Epoch 5/500
247/247 - 1s - loss: 0.6545 - accuracy: 0.6321 - val_loss: 0.6486 - val_accuracy: 0.6358 - 1s/epoch - 4ms/step
Epoch 6/500
247/247 - 1s - loss: 0.6534 - accuracy: 0.6332 - val_loss: 0.6521 - val_accuracy: 0.6358 - 1s/epoch - 5ms/step
Epoch 7/500
247/247 - 1s - loss: 0.6534 - accuracy: 0.6328 - val_loss: 0.6500 - val_accuracy: 0.6358 - 1s/epoch - 5ms/step
Epoch 8/500
247/247 - 1s - loss: 0.6520 - accuracy: 0.6339 - val_loss: 0.6519 - val_accuracy: 0.6368 - 1s/epoch - 4ms/step
Epoch 9/500
247/

In [375]:
# tuner.search(
#     (X_train, y_train),
#     validation_data=(X_test, y_test),
# )

In [376]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test, verbose=2)
print("Test set accuracy:", accuracy)

print("TEST")
y_pred = model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)
print("Accuracy:", accuracy_score(y_test, y_pred_classes))
print("Confusion matrix:\n", confusion_matrix(y_test, y_pred_classes))
print("Classification report:\n", classification_report(y_test, y_pred_classes))

print("TRAIN")
y_pred = model.predict(X_train)
y_pred_classes = (y_pred > 0.5).astype(int)
print("Accuracy:", accuracy_score(y_train, y_pred_classes))
print("Confusion matrix:\n", confusion_matrix(y_train, y_pred_classes))
print("Classification report:\n", classification_report(y_train, y_pred_classes))

62/62 - 0s - loss: 0.6456 - accuracy: 0.6409 - 183ms/epoch - 3ms/step
Test set accuracy: 0.6408700346946716
TEST
62/62 [==============================] - 0s 3ms/step
Accuracy: 0.6408700050581689
Confusion matrix:
 [[1220   37]
 [ 673   47]]
Classification report:
               precision    recall  f1-score   support

           0       0.64      0.97      0.77      1257
           1       0.56      0.07      0.12       720

    accuracy                           0.64      1977
   macro avg       0.60      0.52      0.45      1977
weighted avg       0.61      0.64      0.54      1977

TRAIN
247/247 [==============================] - 0s 1ms/step
Accuracy: 0.6422064777327935
Confusion matrix:
 [[4869  134]
 [2694  207]]
Classification report:
               precision    recall  f1-score   support

           0       0.64      0.97      0.77      5003
           1       0.61      0.07      0.13      2901

    accuracy                           0.64      7904
   macro avg       0.63      0

In [377]:
# # Make predictions on the test set
# y_pred = model.predict(X_test)
# y_pred_classes = (y_pred > 0.5).astype(int)
#
# # Print the predicted and actual labels
# print("Predicted labels:", y_pred_classes.flatten())
# print("Actual labels:", y_test)

In [378]:
# # test with new data
#
# new_data = [
#   (["dominican pop", "merengue", "merengue tipico", "tropical"],
#    ["blues rock", "country rock", "lounge"])
# ]
#
# labels = [
#
# ]
#
# new_df = pd.DataFrame(new_data, columns=["genres", "favourite_genres"])
#
# # Combine genres and favourite_genres
# all_new_genres = list(new_df['genres'] + new_df['favourite_genres'])
#
# # One-hot encode the genres using the previously fit MultiLabelBinarizer (mlb)
# encoded_new_genres = mlb.transform(all_new_genres)
#
# # Split encoded_new_genres into genres and favourite_genres
# encoded_new_genres1 = encoded_new_genres[:, :len(new_df['genres'][0])]
# encoded_new_genres2 = encoded_new_genres[:, len(new_df['genres'][0]):]
#
# # Concatenate the one-hot encoded columns
# X_new = np.concatenate([encoded_new_genres1, encoded_new_genres2], axis=1)
#
# y_new_pred = model.predict(X_new)
# y_new_pred_classes = (y_new_pred > 0.5).astype(int)
#
# # Print the predicted labels
# print("Predicted labels:", y_new_pred_classes.flatten())
# print("Actual labels:", labels)
